In [1]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

loader = DirectoryLoader("C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\upload", glob="*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()
print('.....document_loaded.....')

.....document_loaded.....


In [2]:
import os

AZURE_OPENAI_ENDPOINT="https://azureopenai16.openai.azure.com"
AZURE_OPENAI_API_KEY="75db73a3b9da40b0b6e0e98273a6029f"

In [3]:
from langchain_openai.embeddings import AzureOpenAIEmbeddings
import chromadb

storage_path = "../vectordb"

chroma_client = chromadb.PersistentClient(storage_path)
collection = "test"
chroma_vector_search_index_name = "vector_index"
EMBEDDING_MODEL = "ada0021_6"

#embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
azure_embeddings = AzureOpenAIEmbeddings(
    openai_api_type="azure",
    openai_api_version="2024-02-01",
    openai_api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    model=EMBEDDING_MODEL,
    allowed_special={'<|endoftext|>'}
)

In [4]:
chunk_size = 1000
chunk_overlap = int(0.15*chunk_size)

In [5]:
from rag.chunking import fixed_token_split

splits = fixed_token_split(docs, chunk_size, chunk_overlap)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
splits

[Document(metadata={'source': 'C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\upload\\phoenix.pdf', 'page': 1}, page_content='The Phoenix Project'),
 Document(metadata={'source': 'C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\upload\\phoenix.pdf', 'page': 3}, page_content='The Phoenix Project\nA Novel About IT, DevOps,  \nand Helping Y our Business Win\nGene Kim, Kevin Behr & George Spafford'),
 Document(metadata={'source': 'C:\\Users\\Hammer\\PycharmProjects\\llm_adons\\data\\upload\\phoenix.pdf', 'page': 4}, page_content='The Phoenix Project:  \nA Novel About IT , DevOps, and Helping Y our Business Win\n© 2013 Gene Kim, Kevin Behr & George Spafford\nThe Phoenix Project Resouce Guide \n© 2014 Gene Kim\nAll rights reserved.\nISBN13: 978-0-9882625-0-8\nIT Revolution Press\nPortland, Oregon\ninfo@itrevolution.net\nFor ordering copies of this book: orders@itrevolution.net\nPrinted in the United States of America\nCover design by Joy Stauber\nCover illustration by eboy\nBook i

In [7]:
from rag.ingestion import create_vector_store
from langchain_openai.chat_models import AzureChatOpenAI

vector_store = create_vector_store(splits)

C:\Users\Hammer\Anaconda3\envs\llama\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\Hammer\Anaconda3\envs\llama\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Added 366 chunks to chroma db


In [16]:
from rag.retriever import naiveRetriever

retriever = naiveRetriever(storage_path, azure_embeddings)
print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [26]:
from langchain_core.prompts import PromptTemplate
import re
from langchain.chains import (
    ConversationalRetrievalChain,
    SequentialChain,
    TransformChain,
)

transform = TransformChain(
    input_variables=["input"],
    output_variables=["question", "chat_history"],
    transform=lambda inputs: {
        "question": inputs["input"],
        "chat_history": [
            (human.group(1).strip(), ai.group(1).strip())
            for human, ai in zip(
                re.compile(r'Human: (.*(?:\n(?!(Human|AI):).*)*)').finditer(inputs["history"]),
                re.compile(r'AI: (.*(?:\n(?!(Human|AI):).*)*)').finditer(inputs["history"]),
            )
        ],
    },
)

In [32]:
llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_version="2024-02-01",
    openai_api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    model="gpt35turbo",
    temperature=0
)

In [33]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", input_key='question', return_messages=True, output_key='answer')


qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        verbose=True,
        memory=memory
    )

In [34]:
chat = SequentialChain(
    memory=ConversationBufferMemory(),
    input_variables=["input"],
    output_variables=["answer"],
    chains=[transform, qa],
)

In [37]:
print(chat.run("Who is Laura Beck?"))



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What did the author say about Devops?
Assistant: I'm sorry, I need more context to answer your question. Who is the author and in what context did they mention DevOps?
Human: Who is Bill Palmer?
Assistant: I'm sorry, I cannot answer your question without additional context. Could you please provide more information about where the author mentioned DevOps and Bill Palmer?
Follow Up Input: Who is Laura Beck?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Human: Could you please provi